In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

### Toy problem

A 3D parabola as a toy problem for our neural network to solve 

\begin{equation}
y = x_1^2+x_2^2
\end{equation}


In [ ]:
# obtaining data points
x1 = np.arange(-1,1,0.05)
x2 = np.arange(-1,1,0.05)
X1,X2 = np.meshgrid(x1,x2)
Y = X1**2+X2**2

In [ ]:
# 3D plot for visualizing function
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
surface = ax.plot_surface(X1, X2, Y)

#### Regression

We can use this data as is as a regression problem,

In [ ]:
# flattening data
x1 = X1.flatten()
x2 = X2.flatten()
y = Y.flatten()

In [ ]:
# Setting up the data for processing by DNN
X = np.stack((x1,x2))
Yr = y.reshape(y.shape[0],1).T

#### Binary Classification

We can adapt this into a binary classification problem by thresholding the data,

\begin{equation}
Y_c = 
\begin{cases} 
      0 & y \leq threshold \\
      1 & y > threshold 
\end{cases}
\end{equation}

In [ ]:
# thresholding
Yc = Yr.copy()
Yc[abs(Yc)>=0.5] = 1
Yc[abs(Yc)<0.5] = 0

In [ ]:
# plotting classes
_x1 = x1.reshape(1,x1.shape[0])
_x2 = x2.reshape(1,x2.shape[0])

c11 = _x1[Yc == 0]
c12 = _x2[Yc == 0]

c21 = _x1[Yc == 1]
c22 = _x2[Yc == 1]

plt.figure()
plt.scatter(c11,c12,c='b',s=10)
plt.scatter(c21,c22,c='r',s=10)

#### Multinomial classification

We can adapt this into a multinomial classification problems by adding additional conditions,

\begin{equation}
Y_m = 
\begin{cases} 
      0, & x_1 \leq 0\ and\ Y_c = 0\\
      1, & x_1 > 0\ and\ Y_c = 0\\
      2, & x_1 \leq 0\ and\ Y_c = 1\\
      3, & x_1 > 0\ and\ Y_c = 1\\ 
\end{cases}
\end{equation}

In [ ]:
# thresholding
_Ym = Yc.copy()
_Ym[(_x1 <= 0)*(Yc == 0)] = 0
_Ym[(_x1 > 0)*(Yc == 0)] = 1
_Ym[(_x1 <= 0)*(Yc == 1)] = 2
_Ym[(_x1 > 0)*(Yc == 1)] = 3

In [ ]:
# plotting classes
_x1 = x1.reshape(1,x1.shape[0])
_x2 = x2.reshape(1,x2.shape[0])

c11 = _x1[_Ym == 0]
c12 = _x2[_Ym == 0]

c21 = _x1[_Ym == 1]
c22 = _x2[_Ym == 1]

c31 = _x1[_Ym == 2]
c32 = _x2[_Ym == 2]

c41 = _x1[_Ym == 3]
c42 = _x2[_Ym == 3]

plt.figure()
plt.scatter(c11,c12,c='b',s=10)
plt.scatter(c21,c22,c='r',s=10)
plt.scatter(c31,c32,c='g',s=10)
plt.scatter(c41,c42,c='y',s=10)

In [ ]:
# one-hot encoding
def one_hot_encode(y):
    result = (y == 0)*1
    for i in range(1, int(np.max(y)+1)):
        result = np.append(result, (y == i)*1, axis = 0)
    return result

In [ ]:
# one-hot encoding for NN
Ym = one_hot_encode(_Ym)

### Vanilla neural network

In [ ]:
import numpy as np
from meik.utils.losses import *
from meik.layers import Layer
from meik.metrics import Metrics
from meik.normalizers import Normalizer

class Sequential:
    
    def __init__(self):
        
        self.layers = []
        self.params = {
            'learning_rate': None,
            'epochs': None,
            'loss': None,
        }
        
        self.cost = []
        self.accuracy = []
        
    def add(self,layer):
        
        assert (issubclass(type(layer), Layer))
        
        _id = len(self.layers)
        if _id == 0:
            assert(type(layer.inputs) == int), "Provide number of inputs for initial layer"
            inputs = layer.inputs
        else:
            inputs = self.layers[-1].units
            
        layer.init(_id, inputs)
        self.layers.append(layer)
            
    def build(self, loss = None, normalization = 'none', learning_rate = 0.01, train_metrics = ['default'], eval_metrics = ['default'], thresholds = np.array([0.5])):
        
        self.params['loss'] = loss
        self.params['learning_rate'] = learning_rate
        self.params['normalization'] = normalization
        
        self.normalize = Normalizer(method = normalization)
        
        self.metrics = Metrics(loss = loss, train_metrics = train_metrics, eval_metrics = eval_metrics, thresholds = thresholds)

        # TO DO: proper optimizer objects passed to layer
        for i in range(len(self.layers)):
            self.layers[i].learning_rate = learning_rate
    
    def predict(self,X):
        
        layers = self.layers
        
        A = X
        for i in range(len(layers)):
            A = layers[i].forwardprop(A)
        
        return A

    def backprop(self, Y, A):
        
        layers = self.layers
        
        dZ = A - Y
        dA = self.layers[-1].backprop_output(dZ)
        for i in range(len(layers)-2,-1,-1):
            dA = layers[i].backprop(dA)
            layers[i].update()
            
    def train(self, X, Y, epochs=1, verbose=1):
        
        layers = self.layers
        
        X_norm = self.normalize.train(X)
        
        for i in range(epochs):
            
            A = self.predict(X_norm)
            self.backprop(Y, A)
            
            cost = self.metrics.train(Y, A)
            self.cost.append(cost)
            
            if verbose == 1:
                self.metrics.train_print(i,epochs)
            
        print("------------ Final performance ------------\n")
        self.metrics.train_print(i,epochs)
        
    def evaluate(self, X, Y):
        
        X = self.normalize.evaluate(X)
            
        A = self.predict(X)
        
        score = self.metrics.evaluate(Y, A)
        
        self.score = score
        
        return score
        

### Binary Classification NN

In [ ]:
#import numpy as np

In [ ]:
from meik.models import Sequential
from meik.layers import Layer
from meik.layers import Dense

In [ ]:
model = Sequential()

In [ ]:
model.add(Dense(units=32, activation='relu', initialization='he_uniform', regularization = 'l2', inputs=2))
model.add(Dense(units=8, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

In [ ]:
metrics = ['binary_crossentropy', 'accuracy', 'precision', 'sensitivity', 'specificity', 'confusion_matrix', 'AUC', 'ROC']
th = np.arange(0.0,1.0,0.1)

In [ ]:
model.build(loss='binary_crossentropy', normalization='none', learning_rate=1., eval_metrics = metrics, thresholds = th)

In [ ]:
model.train(X,Yc,epochs=100,verbose=0)

In [ ]:
model.cost[0]

In [ ]:
score = model.evaluate(X,Yc)

In [ ]:
#A = model.predict(X)
#from meik.metrics import metrics_binary_classification
#ev = metrics_binary_classification(metrics = ['accuracy', 'binary_crossentropy', 'precision', 'sensitivity', 'specificity', 'confusion_matrix', 'AUC', 'ROC'], prediction_thresholds = np.arange(0.0,1.0,0.1))
#a = ev.evaluate(Y,A)

### Multinomial NN

In [ ]:
#from meik.models import Sequential
from meik.layers import Layer
from meik.layers import Dense

In [ ]:
model = Sequential()

In [ ]:
model.add(Dense(units=64, activation='relu', regularization='l2', inputs=2))
model.add(Dense(units=64, activation='relu', regularization='l2'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=4, activation='softmax'))

In [ ]:
metrics = ['accuracy', 'binary_crossentropy', 'precision', 'sensitivity', 'specificity', 'confusion_matrix', 'AUC']
th = np.arange(0.0,1.0,0.1)

In [ ]:
model.build(loss='categorical_crossentropy', normalization='normal', learning_rate=0.01, eval_metrics = metrics, thresholds = th)

In [ ]:
model.train(X,Ym,epochs=1000,verbose=0)

In [ ]:
score = model.evaluate(X, Ym)

### Regression NN

In [ ]:
#from meik.models import Sequential
from meik.layers import Layer
from meik.layers import Dense

In [ ]:
model = Sequential()

In [ ]:
model.add(Dense(units=64, activation='relu', regularization='l2', reg_lambda=0.001, inputs=2))
model.add(Dense(units=32, activation='relu', regularization='l1', reg_lambda=0.001))
model.add(Dense(units=8, activation='relu'))
model.add(Dense(units=1, activation='linear'))

In [ ]:
model.build(loss='mse', learning_rate=0.01, eval_metrics = ['mae', 'mse', 'rmse', 'r2'])

In [ ]:
model.train(X,Yr,epochs=1000,verbose=0)

In [ ]:
model.evaluate(X, Yr)

In [ ]:
A = model.predict(X)
np.sum((A >= 0.5) == Yc)

In [ ]:
# residual plots?
plt.figure(1)
plt.scatter(Y.T, (Yr.T - A.T))

### Training with Keras

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [ ]:
model = Sequential()
model.add(Dense(32, input_dim=2, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [ ]:
model.fit(X.T, Yc.T, epochs=500)